In [1]:
import torch
from torch.autograd import Variable
import shap
import numpy
import pandas
import argparse
import os 
from typing import Any, Dict, List
import pandas as pd
from torch.nn import MSELoss, L1Loss
import numpy as np
import json 
import random
from tqdm import tqdm
from pathlib import Path
from scipy.stats import binned_statistic
import matplotlib.pylab as plt


import torch

import torch_frame
from torch.nn import Linear, Module
from torch_frame.data import Dataset
from torch_frame.data import DataLoader
from torch_frame import TensorFrame, stype
from torch_frame.nn.models.ft_transformer import FTTransformer
from torch.nn import Linear, Module, ModuleList
from typing import Any, Dict, List
from torch import Tensor
import torch 

import torch_frame
from torch_frame import TensorFrame, stype
from torch_frame.data.stats import StatType
from torch_frame.nn.conv import TabTransformerConv, FTTransformerConvs
from torch_frame.nn.encoder import (
    EmbeddingEncoder,
    LinearEncoder,
    LinearEmbeddingEncoder,
    StypeWiseFeatureEncoder,
    MultiCategoricalEmbeddingEncoder,
    TimestampEncoder,
)

import sys
sys.path.append('../modeling/')

from model import TabTransformer, stype_encoder_dict_2, stype_encoder_dict_3
from train import get_args_parser

In [2]:
target = "price"
dense_features = ["LON", 
                "LAT", 
                "building_sqft", 
                "Lot Size", 
                "Year Built", 
                "Garage Number", 
                "Bedrooms", 
                "Baths", 
                "Maintenance Fee", 
                "Tax Rate", 
                "Recent Market Value", 
                "Recent Tax Value", 
                 "index"]
cate = ["status", "Property Type", "County", "Private Pool", "Area Pool"]
time_col = ["date"]
cate_multi = ["Foundation_multiclass", "Garage Types_multiclass", 
            "Roof Type_multiclass", "Pool_feature_multiclass", "floor_type_multiclass", 
            "finance_option_multiclass", "Exterior Type_multiclass", "Style_multiclass"]

add_numerial_features = ["elementary_school_star", "middle_school_star", "high_school_star"]
add_cate_multi = ["school_org"] # ["school_names", "school_grades", "school_org"]
text_embedding_col = ["general_desc_roberta_mean_features"]
img_embedding_col = ["img_emb"]
raw_df = pd.read_pickle("../data/property_structured_12162024.pkl").reset_index(drop=True)
raw_df["index"] = np.arange(len(raw_df))
raw_df["date"] = raw_df["date"].apply(lambda x: x.replace("_", "-"))
dense_features = dense_features + add_numerial_features
cate_multi = cate_multi + add_cate_multi

for feat in dense_features:
    raw_df[feat] = raw_df[feat].fillna(raw_df[feat].mean())

col_to_stype = {}
col_to_stype.update({d: stype.numerical for d in dense_features})
col_to_stype.update({d: stype.timestamp for d in time_col})
col_to_stype.update({target: stype.numerical})
col_to_stype.update({d: stype.categorical for d in cate})
col_to_stype.update({d: stype.multicategorical for d in cate_multi})
raw_df = raw_df[dense_features + cate + cate_multi + time_col + [target]]
stype_encoder_dict = stype_encoder_dict_2

for col in cate_multi:
    raw_df[col] = raw_df[col].apply(lambda d: d if isinstance(d, list) else [])

dataset = Dataset(
    raw_df, 
    col_to_stype=col_to_stype,
    target_col="price"
)
dataset.materialize()
torch.manual_seed(10)
dataset.shuffle()
train_dataset, val_dataset = dataset[:0.8], dataset[0.80:]

In [3]:
train_loader = DataLoader(train_dataset.tensor_frame, 
                          batch_size=1024,
                          shuffle=True)
train_iter = iter(train_loader)
train_loader.tensor_frame

TensorFrame(
  num_cols=30,
  num_rows=169568,
  numerical (15): ['Baths', 'Bedrooms', 'Garage Number', 'LAT', 'LON', 'Lot Size', 'Maintenance Fee', 'Recent Market Value', 'Recent Tax Value', 'Tax Rate', 'Year Built', 'building_sqft', 'elementary_school_star', 'high_school_star', 'middle_school_star'],
  timestamp (1): ['date'],
  categorical (5): ['Area Pool', 'County', 'Private Pool', 'Property Type', 'status'],
  multicategorical (9): ['Exterior Type_multiclass', 'Foundation_multiclass', 'Garage Types_multiclass', 'Pool_feature_multiclass', 'Roof Type_multiclass', 'Style_multiclass', 'finance_option_multiclass', 'floor_type_multiclass', 'school_org'],
  has_target=True,
  device='cpu',
)

In [4]:
def load_argparse(filename):
    with open(filename, "r") as f:
        args = json.load(f)
    return args

args = load_argparse("../model_checkpoints/tabformer_t90/config.cfg")
args

model = TabTransformer(
        channels=args["tabformer_channels"],
        num_layers=args["tabformer_num_layers"],
        num_heads=args["tabformer_num_heads"],
        col_stats=train_dataset.col_stats,
        col_names_dict=train_dataset.tensor_frame.col_names_dict,
        stype_encoder_dict=stype_encoder_dict, 
).to("cuda:0")

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) 
loss_fn = MSELoss()

In [6]:
model.state_dict()["encoder.encoder_dict.numerical.weight"][0]

tensor([-0.0123,  0.0067, -0.0096, -0.0017, -0.0086, -0.0094, -0.0008, -0.0128,
         0.0118, -0.0026,  0.0017,  0.0152, -0.0071, -0.0064, -0.0032, -0.0096,
        -0.0081, -0.0018,  0.0039, -0.0175,  0.0027,  0.0106,  0.0100, -0.0031,
         0.0127,  0.0085,  0.0023,  0.0105,  0.0239,  0.0083, -0.0177,  0.0051],
       device='cuda:0')

In [7]:
losses = []

In [13]:
# with autograd.detect_anomaly():
for i in range(10):
    tf = next(train_iter)
    tf = tf.to("cuda:0", non_blocking=True)
    optimizer.zero_grad()
    pred = model(tf)
    # print(pred, tf.y)
    loss = loss_fn(pred, tf.y)
    # print(loss)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    losses[-1]

In [14]:
model.state_dict()["encoder.encoder_dict.numerical.weight"][0]

tensor([-0.0106,  0.0053, -0.0115, -0.0008, -0.0102, -0.0086, -0.0018, -0.0134,
         0.0118, -0.0039,  0.0001,  0.0147, -0.0085, -0.0060, -0.0021, -0.0106,
        -0.0093, -0.0005,  0.0037, -0.0166,  0.0044,  0.0126,  0.0109, -0.0034,
         0.0146,  0.0094,  0.0032,  0.0109,  0.0222,  0.0087, -0.0173,  0.0034],
       device='cuda:0')

In [19]:
train_tf = train_dataset.tensor_frame

In [21]:
feat = train_tf.feat_dict[train_tf.stypes[0]]

In [23]:
torch.max(feat)

tensor(nan)

In [30]:
x = torch.Tensor([[1, 2, 3]])
y = torch.Tensor([[1, 1], [2, 2], [3, 3]])
print(x.shape, y.shape)
torch.einsum("ij,jk->ijk", x, y)

torch.Size([1, 3]) torch.Size([3, 2])


tensor([[[1., 1.],
         [4., 4.],
         [9., 9.]]])

In [31]:
tf

TensorFrame(
  num_cols=30,
  num_rows=16,
  numerical (15): ['Baths', 'Bedrooms', 'Garage Number', 'LAT', 'LON', 'Lot Size', 'Maintenance Fee', 'Recent Market Value', 'Recent Tax Value', 'Tax Rate', 'Year Built', 'building_sqft', 'elementary_school_star', 'high_school_star', 'middle_school_star'],
  timestamp (1): ['date'],
  categorical (5): ['Area Pool', 'County', 'Private Pool', 'Property Type', 'status'],
  multicategorical (9): ['Exterior Type_multiclass', 'Foundation_multiclass', 'Garage Types_multiclass', 'Pool_feature_multiclass', 'Roof Type_multiclass', 'Style_multiclass', 'finance_option_multiclass', 'floor_type_multiclass', 'school_org'],
  has_target=True,
  device='cuda:0',
)

In [46]:
tf = next(train_iter)

In [57]:
col_names = model.encoder.col_names_dict[stype]

In [79]:
stype = tf.stypes[0]
feat = tf.feat_dict[stype].to("cuda:0")
linear_encoder = model.encoder.encoder_dict[stype.value]
# model.encoder.encoder_dict[stype.value](feat[2:3], col_names)

In [29]:
## Get the validation set address

In [30]:
raw_df = pd.read_pickle("../data/property_structured_12162024.pkl").reset_index(drop=True)
raw_df["index"] = np.arange(len(raw_df))
val_tf = val_dataset.tensor_frame
stype = val_tf.stypes[0]
val_tf.col_names_dict[stype]

['Baths',
 'Bedrooms',
 'Garage Number',
 'LAT',
 'LON',
 'Lot Size',
 'Maintenance Fee',
 'Recent Market Value',
 'Recent Tax Value',
 'Tax Rate',
 'Year Built',
 'building_sqft',
 'elementary_school_star',
 'high_school_star',
 'index',
 'middle_school_star']

In [31]:
val_indices = val_tf.feat_dict[stype][:, -2].clone().numpy().astype(np.int)
val_indices

/tmp/ipykernel_2325/2765142923.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  val_indices = val_tf.feat_dict[stype][:, -2].clone().numpy().astype(np.int)


array([169568, 169569, 169570, ..., 211957, 211958, 211959])

In [32]:
s1 = set(raw_df["index"].values)
s2 = set(val_indices)

In [37]:
# raw_df[raw_df["index"].apply(lambda x: x in s2)].drop(["index"], axis=1).to_pickle("../data/property_structured_12162024_validation.pkl")

42392